https://machinelearningmastery.com/learn-add-numbers-seq2seq-recurrent-neural-networks/

In [9]:
from random import seed
from random import randint
from numpy import array
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from sklearn.metrics import mean_squared_error

In [3]:
seed(1)
X, y = list(), list()
for i in range(100):
    in_pattern = [randint(1,100) for _ in range(2)]
    out_pattern = sum(in_pattern)
    if i < 5:
        print(in_pattern, out_pattern)
    X.append(in_pattern)
    y.append(out_pattern)

[18, 73] 91
[98, 9] 107
[33, 16] 49
[64, 98] 162
[58, 61] 119


In [6]:
# generate examples of random integers and their sum
def random_sum_pairs(n_examples, n_numbers, largest):
    X, y = list(), list()
    for i in range(n_examples):
        in_pattern = [randint(1, largest) for _ in range(n_numbers)]
        out_pattern = sum(in_pattern)
        X.append(in_pattern)
        y.append(out_pattern)
    # format as NumPy arrays
    X,y = array(X), array(y)
    # normalize
    X = X.astype('float') / float(largest * n_numbers)
    y = y.astype('float') / float(largest * n_numbers)
    return X, y

In [7]:
# invert normalization
def invert(value, n_numbers, largest):
    return round(value * float(largest * n_numbers))

#### Model

In [11]:
n_examples = 100
n_numbers = 2
largest = 100
# define LSTM configuration
n_batch = 1
n_epoch = 100

In [10]:
model = Sequential()
model.add(LSTM(6, return_sequences=True))
model.add(LSTM(6))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

In [12]:
# train LSTM
for _ in range(n_epoch):
    X, y = random_sum_pairs(n_examples, n_numbers, largest)
    X = X.reshape(n_examples, n_numbers, 1)  # add 1 dimension
    model.fit(X, y, epochs=1, batch_size=n_batch, verbose=2)

Epoch 1/1
 - 2s - loss: 0.1705
Epoch 1/1
 - 1s - loss: 0.0491
Epoch 1/1
 - 1s - loss: 0.0309
Epoch 1/1
 - 1s - loss: 0.0238
Epoch 1/1
 - 1s - loss: 0.0301
Epoch 1/1
 - 1s - loss: 0.0187
Epoch 1/1
 - 1s - loss: 0.0157
Epoch 1/1
 - 1s - loss: 0.0132
Epoch 1/1
 - 1s - loss: 0.0090
Epoch 1/1
 - 1s - loss: 0.0073
Epoch 1/1
 - 1s - loss: 0.0066
Epoch 1/1
 - 1s - loss: 0.0046
Epoch 1/1
 - 1s - loss: 0.0057
Epoch 1/1
 - 1s - loss: 0.0033
Epoch 1/1
 - 1s - loss: 0.0034
Epoch 1/1
 - 1s - loss: 0.0031
Epoch 1/1
 - 1s - loss: 0.0035
Epoch 1/1
 - 1s - loss: 0.0030
Epoch 1/1
 - 1s - loss: 0.0024
Epoch 1/1
 - 1s - loss: 0.0022
Epoch 1/1
 - 1s - loss: 0.0021
Epoch 1/1
 - 1s - loss: 0.0019
Epoch 1/1
 - 1s - loss: 0.0016
Epoch 1/1
 - 1s - loss: 0.0013
Epoch 1/1
 - 1s - loss: 0.0010
Epoch 1/1
 - 1s - loss: 8.2899e-04
Epoch 1/1
 - 1s - loss: 9.1822e-04
Epoch 1/1
 - 1s - loss: 7.7952e-04
Epoch 1/1
 - 1s - loss: 5.6560e-04
Epoch 1/1
 - 1s - loss: 7.1647e-04
Epoch 1/1
 - 1s - loss: 4.7454e-04
Epoch 1/1
 - 1s

In [15]:
# evaluate on some new patterns
X, y = random_sum_pairs(n_examples, n_numbers, largest)
X = X.reshape(n_examples, n_numbers, 1)
result = model.predict(X, batch_size=n_batch, verbose=0)
# calculate error
expected = [invert(x, n_numbers, largest) for x in y]
predicted = [invert(x, n_numbers, largest) for x in result[:,0]]
rmse = sqrt(mean_squared_error(expected, predicted))
print('RMSE: %f' % rmse)
# show some examples
for i in range(20):
    error = expected[i] - predicted[i]
    print('Expected=%d, Predicted=%d (err=%d)' % (expected[i], predicted[i], error))

RMSE: 0.806226
Expected=140, Predicted=140 (err=0)
Expected=121, Predicted=120 (err=1)
Expected=47, Predicted=46 (err=1)
Expected=98, Predicted=98 (err=0)
Expected=69, Predicted=70 (err=-1)
Expected=105, Predicted=104 (err=1)
Expected=134, Predicted=135 (err=-1)
Expected=67, Predicted=66 (err=1)
Expected=141, Predicted=141 (err=0)
Expected=69, Predicted=69 (err=0)
Expected=27, Predicted=27 (err=0)
Expected=92, Predicted=92 (err=0)
Expected=101, Predicted=101 (err=0)
Expected=192, Predicted=190 (err=2)
Expected=197, Predicted=195 (err=2)
Expected=72, Predicted=71 (err=1)
Expected=47, Predicted=46 (err=1)
Expected=144, Predicted=143 (err=1)
Expected=147, Predicted=146 (err=1)
Expected=87, Predicted=87 (err=0)


That is, the order of the input no longer matters. We could shuffle it up any way we want and still learn the problem.

MLPs are designed to learn mapping functions and can easily nail the problem of learning how to add numbers.